### Setup



In [ ]:
#PLS imports
from sys import stdout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np; print('numpy Version:', np.__version__)
import pandas as pd; print('pandas Version:', pd.__version__)
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter;
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)


### Reading Dataset

In [ ]:
data_cleaned = pd.read_csv('SQL_extracted datasets/Formatted_data/FINALlanghill_mir_extractready_revised_7pm_mir_aligned_20210711.csv', sep=',')
print(data_cleaned.shape) #Shape is 417657 rows and 18 columns


### External Dataset

In [ ]:
X = data_cleaned.iloc[:, 14:1074]
y = data_cleaned['weeklyave_FI']
X2 = savgol_filter(X, 17, polyorder=2, deriv=2)
X1 = savgol_filter(X, 17, polyorder=2, deriv=1)
#Alternative split method which will randomly split the data
from sklearn.model_selection import train_test_split

test_size = 0.1


X_test, X_train, y_test, y_train =train_test_split(X1,y, train_size=0.1, random_state = 11)
#_train, X_valid, y_train, y_valid = train_test_split(X_rem,y_rem, train_size=0.8)
# check dimensions
print('X_train: ', X_train.shape,  'y_train: ', y_train.shape)
#print('X_validation', X_valid.shape, 'y_validation: ', y_valid.shape)
print('X_test', X_test.shape, 'y_test: ', y_test.shape)


# check the proportions
total = X_train.shape[0]  + X_test.shape[0]
print('X_train proportion:', X_train.shape[0] / total)
#print('X_validation proportion:', X_valid.shape[0] / total)
print('X_test proportion:', X_test.shape[0] / total)

### PLS Prep

In [ ]:
y = y_train
X = X_train
y.shape
X.shape
wl = np.arange(900, 5140, 4)

print(len(wl))
1060
fig, ax = plt.subplots()
ax.plot(wl, np.mean(X, axis = 0),   label = "Mean value", alpha = 1)
ax.plot(wl, np.percentile(X, 99, axis = 0),  label = "99th percentile", alpha = .35)
ax.plot(wl, np.percentile(X, 1, axis = 0), label = "1st percentile",  alpha = .35)
#ax.plot(y_test, intercept + slope * y_test, label=line)

plt.title('Untreated Spectrum')
ax.set_xlabel('Wavenumber (cm$^{-1}$)')
ax.set_ylabel('Absorbance')
ax.legend(facecolor='white')
plt.savefig('output3.png', dpi=250)
plt.show()
#fig.tight_layout()


fig, ax = plt.subplots()
ax.plot(wl, np.mean(X1, axis = 0), label = "Mean value",  alpha = 1)
ax.plot(wl, np.percentile(X1, 99, axis = 0), label = "99th percentile", alpha = .35)

ax.plot(wl, np.percentile(X1, 1, axis = 0), label = "1st percentile",  alpha = .35)

#ax.plot(y_test, intercept + slope * y_test, label=line)

plt.title('First Derivative Spectrum')
ax.set_xlabel('Wavenumber (cm$^{-1}$)')
ax.set_ylabel('Absorbance')
ax.legend(facecolor='white')
fig.tight_layout()
plt.savefig('output6.png', dpi=250)

plt.show()
# Plot the data absorption at each datapoint - need to clarify the range & unit of wavelengths 


with plt.style.context('ggplot'):
    plt.plot(wl, X.T)
    plt.xlabel("Datapoint - some wavelength value")
    plt.ylabel("Absorbance")
https://www.kaggle.com/phamvanvung/partial-least-squares-regression-in-python
#X2 = savgol_filter(X, 17, polyorder=2, deriv=2)
X1 = savgol_filter(X, 17, polyorder=2, deriv=1)

X1.shape
plt.figure(figsize=(8, 4.5))
with plt.style.context('ggplot'):
    plt.plot(wl, X1.T)
    plt.xlabel("Wavelengths (nm)")
    plt.ylabel("D1 Absorbance")
    plt.show()
plt.figure(figsize=(8, 4.5))
with plt.style.context('ggplot'):
    plt.plot(wl, X2.T)
    plt.xlabel("Wavelengths (nm)")
    plt.ylabel("D2 Absorbance")
    plt.show()
def optimise_pls_cv(X1, y, n_comp, plot_components = True):
    # Define PLS object
    pls = PLSRegression(n_components=n_comp)

    # Cross-validation
    y_cv = cross_val_predict(pls, X1, y, cv=10)

    # Calculate scores
    r2 = r2_score(y, y_cv)
    mse = mean_squared_error(y, y_cv)
    rpd = y.std()/np.sqrt(mse)
    
    return (y_cv, r2, mse, rpd)
%%time

r2s = []
mses = []
rpds = []
xticks = np.arange(1, 21)
for n_comp in xticks:
    y_cv, r2, mse, rpd = optimise_pls_cv(X, y, n_comp)
    r2s.append(r2)
    mses.append(mse)
    rpds.append(rpd)
CPU times: user 3h 40min 32s, sys: 3h 28min 54s, total: 7h 9min 27s
Wall time: 16min 4s

 # Calculate and print the position of minimum in MSE      
    msemin = np.argmin(mses)      
    print("Suggested number of components: ", msemin+1)      
    stdout.write("\n")
Suggested number of components:  20
import matplotlib.ticker as mticker
# Plot the mses
def plot_metrics(vals, ylabel, objective):
    #with plt.style.context('ggplot'):
    plt.plot(range(1, 21), np.array(vals), '-v')
    if objective=='min':
        idx = np.argmin(vals)
    else:
        idx = np.argmax(vals)
    plt.plot(xticks[idx], np.array(vals)[idx], '*', ms=10, color = "red")

    plt.ticklabel_format(style='plain', axis='x', useOffset=False)
    #plt.plot(range(1, 20), logger.acc)
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.xlabel('Number of PLS components')
    #plt.xticks(np.arange(1, 21,1.0))
    #plt.xticks(range(1,20))
    plt.ylabel(ylabel)
    plt.title('Incremental value of each PLS component')

    plt.show()
fig, ax = plt.subplots()
ax.plot(wl, np.mean(X1, axis = 0), label = "Mean value",  alpha = 1)
ax.plot(wl, np.percentile(X1, 99, axis = 0), label = "99th percentile", alpha = .35)


ax.plot(wl, np.percentile(X1, 1, axis = 0), label = "1st percentile",  alpha = .35)

#ax.plot(y_test, intercept + slope * y_test, label=line)

plt.title('First Derivative Spectrum')
ax.set_xlabel('Wavenumber (cm$^{-1}$)')
ax.set_ylabel('Absorbance')
ax.legend(facecolor='white')


fig.tight_layout()
plt.savefig('output6.png', dpi=250)

plt.show()
fig, (ax1, ax2) = plt.subplots(1, 2)
fig.suptitle('Incremental value of each PLS component')
objective = 'min'
ylabel = 'RMSE'
vals = np.sqrt(mses)

ax1.plot(range(1, 21), np.array(vals), '-v')
if objective=='min':
    idx = np.argmin(vals)
else:
    idx = np.argmax(vals)
ax1.plot(xticks[idx], np.array(vals)[idx], '*', ms=10, color = "red")

ax1.ticklabel_format(style='plain', axis='x', useOffset=False)
#plt.plot(range(1, 20), logger.acc)
#plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
ax1.set_xlabel('Number of PLS components')
#plt.xticks(np.arange(1, 21,1.0))
#plt.xticks(range(1,20))
ax1.set_ylabel(ylabel)


objective = 'max'
ylabel = 'R$^{2}$'
vals =r2s
ax2.plot(range(1, 21), np.array(vals), '-v')
if objective=='min':
    idx = np.argmin(vals)
else:
    idx = np.argmax(vals)
ax2.plot(xticks[idx], np.array(vals)[idx], '*', ms=10, color = "red")

ax2.ticklabel_format(style='plain', axis='x', useOffset=False)
#plt.plot(range(1, 20), logger.acc)
#ax2.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
ax2.set_xlabel('Number of PLS components')
#plt.xticks(np.arange(1, 21,1.0))

#plt.xticks(range(1,20))
ax2.set_ylabel(ylabel)
#fig.tight_layout()
fig.tight_layout(rect=[0, 0.03, 1, 0.95])


plt.savefig('outputPLS.png', dpi=250)


plot_metrics(np.sqrt(mses), 'RMSE', 'min')
plot_metrics(r2s, 'R$^{2}$', 'max')
x
[0, 5, 9, 10, 15]
# Define PLS object with optimal number of components      
pls_opt = PLSRegression(n_components=msemin+1)        
# Fit to the entire dataset   
pls_opt.fit(X, y)     
y_c = pls_opt.predict(X) 

np.std(y)
%%time 


### Cross-Validation

In [ ]:
y_cv = cross_val_predict(pls_opt, X, y, cv=10)       
# Calculate scores for calibration and cross-validation      
score_c = r2_score(y, y_c)      
score_cv = r2_score(y, y_cv)        
# Calculate mean squared error for calibration and cross validation      
mse_c = mean_squared_error(y, y_c)  
rmse_c = np.sqrt(mse_c)
mse_cv = mean_squared_error(y, y_cv)   
rmse_cv = np.sqrt(mse_cv)
print('R2 calib: %5.3f'  % score_c)      
print('R2 CV: %5.3f'  % score_cv)   
print('R calib: %5.3f'  % np.sqrt(score_c)      )
print('R CV: %5.3f'  % np.sqrt(score_cv)  )
print('RMSE calib: %5.3f' % rmse_c)      
print('RMSE CV: %5.3f' % rmse_cv)        

# Plot regression and figures of merit      
rangey = max(y) - min(y)      
rangex = max(y_c) - min(y_c)        
# Fit a line to the CV vs response      
z = np.polyfit(y, y_c, 1)     
with plt.style.context(('ggplot')):      
    fig, ax = plt.subplots(figsize=(9, 5))   
    ax.scatter(y_c, y, c='red', edgecolors='k')        
    #Plot the best fit line        
    ax.plot(np.polyval(z,y), y, c='blue', linewidth=1)     
    #Plot the ideal 1:1 line          
    ax.plot(y, y, color='green', linewidth=1)          
    plt.title('$R^{2}$ (CV): '+str(score_cv))          
    plt.xlabel('Predicted $^{\circ}$Feed Intake')
    plt.ylabel('Measured $^{\circ}$Feed Intake')
    plt.show() 

#Get std of RMSE
ysq = np.sqrt(y)
y_csq = np.sqrt(y_c[:,0])
e = (ysq-y_csq)
sd = np.std(e)

print("Std of RMSE calib: ", sd)

y_cvsq = np.sqrt(y_cv[:,0])
e = (ysq-y_cvsq)
sd = np.std(e)

print("Std of RMSE CV: ", sd)


### Correlations & Measures of Accuracy

In [ ]:
print('Corr: %0.4f, R2: %0.4f, RMSE: %0.4f, RPD: %0.4f' %(corr,r2, rmse, rpd))
preds = pls_opt.predict(X_test) 
preds = preds[:,0]
from scipy.stats import pearsonr
# calculate Pearson's correlation
corr, p = pearsonr(y_test, preds)
print('Pearsons correlation: %.3f' % corr)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
# calculate coeff of determination
r2 = r2_score(y_test, preds)
print('Coeff of determination: %.3f' % r2)


In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, preds))
print('RMSE of prediction: %.3f' % rmse)
